# Generación de Tests Unitarios para Java con RAG

Este notebook conecta contra un índice de Pinecone existente y genera tests a partir de un archivo `.java`.

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

# Ajusta el PYTHONPATH si tu código está en src/
import sys
sys.path.append(os.path.join(os.getcwd(), 'src'))

In [1]:
java_path = input('Ingresa la ruta al archivo .java: ').strip()
use_rag_input = input('Usar RAG? (True/False): ').strip()
use_rag = use_rag_input.lower() in ('true', '1', 'yes')
print(f'Java file: {java_path}, use_rag={use_rag}')

Java file: /home/jburgos/IdeaProjects/paw-2024a-05/persistence/src/main/java/ar/edu/itba/paw/persistence/PostDaoJpa.java, use_rag=True


In [6]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Pinecone as LC_Pinecone
import os

# Configura modelo de embeddings
embedding_model_name = os.getenv('EMBEDDING_MODEL', 'mxbai-embed-large')
embeddings = OllamaEmbeddings(model=embedding_model_name)

# Conecta a Pinecone usando el índice ya creado
index_name = os.getenv('PINECONE_INDEX_NAME', 'java-tests')
namespace = os.getenv('PINECONE_NAMESPACE', 'java_tests')
vectorstore = LC_Pinecone.from_existing_index(
    embedding=embeddings,
    index_name=index_name,
    namespace=namespace
)
print(f'Conectado a Pinecone index "{index_name}" en namespace "{namespace}"')

ValueError: client should be an instance of pinecone.Index, got <class 'pinecone.data.index.Index'>

In [ ]:
from src/model import Model

# Instancia Model con docs dummy (no se usan cuando use_rag=True)
model = Model(docs=[], use_rag=use_rag)
model.vectorstore = vectorstore
print('Modelo inicializado y vectorstore reemplazado.')

In [ ]:
from src/parser import Parser

# Verificar que el archivo existe
if not os.path.isfile(java_path):
    raise FileNotFoundError(f'No se encontró el archivo: {java_path}')

# Genera los tests
test_code = model.generate_test_methods(java_path, k=3)
print('\n=== Test generado ===\n')
print(test_code)

In [ ]:
output_dir = os.getenv('TEST_OUTPUT_DIR', 'generated_tests')
from model import Model as _M
class_name = os.path.splitext(os.path.basename(java_path))[0]
_M.save_test_class(output_dir, test_code, class_name)